In [20]:
import pandas as pd
import clickhouse_driver as ch
from dotenv import load_dotenv
import os
from main import *
from fullCalc import getCampaignById

load_dotenv()

campaign = getCampaignById(host=os.environ.get('HOST'),
                user=os.environ.get('CLICKHOUSE_USERNAME'),
                password=os.environ.get('PASSWORD'),
                campaignId=28639)
df = loadData(host=os.environ.get('HOST'),
                user=os.environ.get('CLICKHOUSE_USERNAME'),
                password=os.environ.get('PASSWORD'),
                campaign_name=campaign)

df['datetime'] = pd.to_datetime(df['datetime']).dt.date

            0
0  RU | Белье


ValueError: Length mismatch: Expected axis has 0 elements, new values have 2 elements

In [22]:
q="""SELECT campaign_id, AVG(bid), AVG(approve), AVG(ctr), AVG(cr), AVG(epc), AVG(ecpm)
FROM (SELECT AVG(IF(cpa_money=0,NULL,cpa_money)) as bid,
			sum(postbacks_confirmed_count)/sum(postbacks_total_count) as approve,	
			sum(clicks)/sum(ad_shows) as ctr,
			sum(postbacks_total_count)/sum(clicks) as cr,
			bid*sum(postbacks_confirmed_count)/sum(clicks) as epc,
			bid*sum(postbacks_confirmed_count)/sum(ad_shows)*1000 as ecpm, ad_id, sum(ad_shows) as shows
		FROM luckyfeed.ad_shows
		WHERE ad_id in (SELECT ad_id 
						FROM ads.ads 
						WHERE campaign_id == 28639)
GROUP BY ad_id) asu2
LEFT JOIN (SELECT ad_id, campaign_name FROM ads.ads) c ON c.ad_id == asu2.ad_id
WHERE bid is not NULL and 
		shows!=0
GROUP BY campaign_id;"""

sqlCH = ch.Client(host=os.environ.get('HOST'),
                user=os.environ.get('CLICKHOUSE_USERNAME'),
                password=os.environ.get('PASSWORD'),)
df = pd.DataFrame(sqlCH.execute(q))
df

""
